### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [3]:
if user_movie_subset.iloc[0,1] != np.nan:
    print(1)
#user_mat
user_mat(~np.isnan(user_mat)).size

1


NameError: name 'user_mat' is not defined

In [4]:
user_mat = np.random.rand(20, 4) # user matrix filled with random values of shape user x latent 
movie_mat = np.random.rand(4, 4)  # movie matrix filled with random values of shape latent x movies
temp_check = user_mat.copy()
for i in range(user_mat[:,0].size):
    for j in range(user_mat[i,:].size):
        user_mat[i,j] = user_mat[i,j] + 2*0.01*(user_movie_subset.iloc[i,j]-np.dot(user_mat[i,:],movie_mat[:,j]))**2*movie_mat[i%4,j] 
        print(str(i)+' '+str(user_mat[i,j]))
        movie_mat[i%4,j] = movie_mat[i%4,j] + 2*0.01*(user_movie_subset.iloc[i,j]-np.dot(user_mat[i,:],movie_mat[:,j]))**2*user_mat[i,j]

#print(user_movie_subset.loc[index,:])
    
#print(user_movie_subset.iloc[0,:])
#user_movie_subset.head()

0 1.13182500302
0 1.67187307118
0 2.01041162199
0 1.2171806898
1 1.21008997516
1 0.674670392656
1 0.881191809741
1 0.939769146671
2 0.796315612278
2 0.423883754045
2 0.0785116686706
2 0.49051543646
3 1.1170367293
3 1.05215959538
3 1.1061738069
3 1.39839953963
4 2.40008055118
4 0.815625675405
4 0.676742297328
4 0.856799474442
5 0.701232954718
5 0.903909605796
5 1.22948025445
5 0.434709797819
6 1.05095516973
6 0.665687500399
6 0.814067691121
6 0.92836820421
7 1.02823532085
7 0.96415437874
7 1.11436542435
7 0.810300414465
8 1.72191973037
8 0.748396684476
8 0.505346293641
8 0.50896002675
9 1.33803033176
9 0.724447585542
9 0.319176086845
9 1.00111103001
10 1.43862388572
10 1.03293342666
10 0.1846316771
10 0.408256098864
11 1.46937068977
11 0.693979145142
11 1.12447503232
11 0.500153970301
12 2.4020886489
12 0.45474390144
12 0.36882060398
12 0.597777834684
13 0.851210529944
13 0.46020248734
13 0.981145124938
13 0.825851059238
14 1.44402543698
14 1.06571412645
14 0.348203493781
14 0.940871119

In [5]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users =len(ratings_mat) # number of rows in the matrix
    n_movies = ratings_mat[0,:].size # number of movies in the matrix
    num_ratings = ratings_mat.size  # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies)  # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for epoch in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                # if the rating exists
                if ratings_mat[i,j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error = ratings_mat[i,j]-np.dot(user_mat[i,:],movie_mat[:,j])
                    sse_accum += error**2
                    #print(np.dot(user_mat[i,:],movie_mat[:,j]))
                    # Keep track of the total sum of squared errors for the matrix                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        
                        user_mat[i,k] +=  2*learning_rate*error*movie_mat[k,j] 
                        movie_mat[k,j] +=  2*learning_rate*error*user_mat[i,k]
            #print(user_mat[i:])

                    
        # print results for iteration
        print("%d \t\t %f" % (epoch+1, sse_accum / num_ratings))
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [7]:
user_mat, movie_mat = FunkSVD(np.matrix(user_movie_subset), latent_features=4, learning_rate=0.005, iters=10) # use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 44.898606
2 		 16.495435
3 		 4.393717
4 		 2.837417
5 		 2.711261
6 		 2.681573
7 		 2.656764
8 		 2.627338
9 		 2.592225
10 		 2.551252


In [8]:
print(movie_mat)
print(user_mat)

[[ 2.22859159  1.77544363  2.75297499  2.41719182]
 [ 1.81182231  2.09745902  1.21020379  2.16565031]
 [ 2.25802457  2.06368205  2.43045957  1.75344551]
 [ 2.39609436  1.75150389  2.56216218  2.37656024]]
[[ 1.14297348  1.25030034  0.76277279  1.49661321]
 [ 1.11133319  0.4871084   0.71074514  1.55580367]
 [ 0.55048263  0.67954506  1.10474467  1.25721422]
 [ 1.04498005  0.74452284  1.27157709  1.28733265]
 [ 0.86093926  0.44883457  1.12339686  1.40434729]
 [ 0.92555126  0.63838791  0.74525271  0.81627424]
 [ 1.18522525  1.46807706  0.91588651  0.75279969]
 [ 0.65191977  0.98305617  1.30297772  0.89451841]
 [ 0.90823282  0.73914706  1.07497331  1.15071224]
 [ 1.11754569  0.68045184  0.52724902  1.13548567]
 [ 1.20374542  0.7990383   1.21358907  0.84284171]
 [ 1.10153922  1.04125532  1.24818236  1.03035492]
 [ 1.30579719  0.98823755  0.69886174  1.22480065]
 [ 0.63204179  0.90120666  0.80691133  0.75300987]
 [ 1.38085412  0.55628957  1.42356325  1.03444427]
 [ 1.61226067  1.1090606   0.8

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [10]:
user_mat, movie_mat = FunkSVD(np.matrix(user_movie_subset), latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 45.197149
2 		 15.953725
3 		 4.399009
4 		 2.955803
5 		 2.811493
6 		 2.772256
7 		 2.750364
8 		 2.732493
9 		 2.715413
10 		 2.698053
11 		 2.679822
12 		 2.660233
13 		 2.638825
14 		 2.615142
15 		 2.588722
16 		 2.559100
17 		 2.525817
18 		 2.488424
19 		 2.446502
20 		 2.399685
21 		 2.347683
22 		 2.290317
23 		 2.227551
24 		 2.159519
25 		 2.086550
26 		 2.009176
27 		 1.928124
28 		 1.844284
29 		 1.758662
30 		 1.672307
31 		 1.586241
32 		 1.501378
33 		 1.418465
34 		 1.338042
35 		 1.260425
36 		 1.185717
37 		 1.113837
38 		 1.044567
39 		 0.977599
40 		 0.912588
41 		 0.849202
42 		 0.787163
43 		 0.726278
44 		 0.666466
45 		 0.607772
46 		 0.550366
47 		 0.494539
48 		 0.440672
49 		 0.389210
50 		 0.340613
51 		 0.295314
52 		 0.253677
53 		 0.215960
54 		 0.182296
55 		 0.152685
56 		 0.127003
57 		 0.105023
58 		 0.086437
59 		 0.070891
60 		 0.058010
61 		 0.047418
62 		 0.038764
63 		 0.031725
64 		

In [11]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.99999913   9.99999948   9.99999983  10.00000093]
 [  9.99999867   3.99999916   8.99999973  10.00000148]
 [  7.99999803   8.99999889   9.99999966   5.00000207]
 [  9.00000033   8.00000021  10.00000011   9.99999969]
 [ 10.00000108   5.00000067   9.00000025   8.99999887]
 [  5.99999792   3.99999873   9.99999961   6.00000228]
 [  9.0000006    8.00000048  10.0000002    8.99999931]
 [  9.99999953   4.9999998    8.99999996   8.00000048]
 [  7.0000007    8.00000043  10.00000017   7.99999929]
 [  9.00000025   5.00000019   9.00000008   6.99999973]
 [  9.00000089   8.00000056  10.0000002    7.99999906]
 [  9.00000113  10.00000061  10.00000021   8.99999886]
 [ 10.0000009    9.00000053  10.00000016   7.99999905]
 [  4.99999883   7.99999926   4.99999975   8.00000127]
 [  9.99999962   7.99999973   9.99999991  10.00000044]
 [  8.99999959   8.99999986   9.99999995  10.00000036]
 [  9.00000094   8.00000051   8.00000017   7.99999905]
 [  9.99999955   7.99999974   0.99999991  10.00000046]
 [  5.0000

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [12]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
#ratings_mat[]

In [14]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(np.matrix(user_movie_subset), latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 45.857427
2 		 16.839381
3 		 4.351687
4 		 2.826556
5 		 2.709972
6 		 2.672437
7 		 2.641627
8 		 2.609094
9 		 2.572579
10 		 2.531137
11 		 2.484128
12 		 2.431059
13 		 2.371618
14 		 2.305750
15 		 2.233728
16 		 2.156214
17 		 2.074280
18 		 1.989376
19 		 1.903241
20 		 1.817747
21 		 1.734704
22 		 1.655668
23 		 1.581770
24 		 1.513626
25 		 1.451330
26 		 1.394528
27 		 1.342535
28 		 1.294491
29 		 1.249486
30 		 1.206674
31 		 1.165343
32 		 1.124951
33 		 1.085138
34 		 1.045716
35 		 1.006651
36 		 0.968030
37 		 0.930032
38 		 0.892893
39 		 0.856873
40 		 0.822226
41 		 0.789177
42 		 0.757900
43 		 0.728505
44 		 0.701036
45 		 0.675469
46 		 0.651724
47 		 0.629666
48 		 0.609127
49 		 0.589914
50 		 0.571818
51 		 0.554632
52 		 0.538151
53 		 0.522181
54 		 0.506543
55 		 0.491076
56 		 0.475639
57 		 0.460110
58 		 0.444388
59 		 0.428391
60 		 0.412060
61 		 0.395357
62 		 0.378266
63 		 0.360791
64 		

In [15]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.000013038176482:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [17]:
first_1000_users = np.matrix(user_by_movie.head(1000))
first_1000_users

matrix([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [18]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat =FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20) #fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 0.008179
2 		 0.003769
3 		 0.002586
4 		 0.001991
5 		 0.001616
6 		 0.001353
7 		 0.001158
8 		 0.001008
9 		 0.000888
10 		 0.000791
11 		 0.000711
12 		 0.000645
13 		 0.000589
14 		 0.000541
15 		 0.000500
16 		 0.000465
17 		 0.000435
18 		 0.000409
19 		 0.000386
20 		 0.000366


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [24]:
 np.count_nonzero(~np.isnan(first_1000_users))

10852

In [22]:
preds = np.dot(user_mat, movie_mat)
count_vals=0
count_nans=0
for i in range(len(preds[:,0])):
    for j in range(len(preds[0,:])):
        if preds[i,j] != np.nan:
            count_vals += 1
        else:
            count_nans += 1

In [ ]:
# Replace each of the comments below with the correct values
num_ratings =  np.count_nonzero(~np.isnan(first_1000_users))# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

In [ ]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")